## Steam Scraping Code
#### 1. Extract Top 100 Games from Steam and SteamSpy
#### 2. Create a master list of Games using the above
#### 3. Extract Game Metadata using SteamSpy API
#### 4. Extract additional Game Metadata using Selenium/BeautifulSoup
#### 5. Extract Concurrent Players, Twitch metrics at Day/Hour level using Selenium/BeautifulSoup

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import urllib
import numpy as np
import requests
import json
from time import sleep
import re
import os
import ast
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm

In [2]:
#Convert Data Type as per input dictionary
def convert_data_type(df,dtype_dict):
    df = df.astype(dtype_dict)
    return df

#Get Top 100 Applications used in last 2 weeks from Steam Spy
def get_top_100_steamspy():
    url = 'http://steamspy.com/api.php?request=top100in2weeks'
    response = urllib.request.urlopen(url).read()
    top_100_steam_spy = json.loads(response.decode())
    top_100_steam_spy_df = pd.DataFrame.from_dict(top_100_steam_spy, orient='index')
    top_100_steam_spy_df.reset_index(drop=True, inplace=True)
    top_100_steam_spy_df.drop(['score_rank','userscore'], axis=1, inplace=True)
    top_100_steam_spy_df.columns = ['App_ID','App_Name','Developer','Publisher','Positive','Negative',
                                    'Owners','Avg_Playtime','Avg_Playtime_2wk','Median_Playtime','Median_Playtime_2wk',
                                    'Current_Price','Initial_Price','Discount']
    
    dtype_dict = {'App_ID': int, 'Positive': int, 'Negative': int, 'Avg_Playtime': int, 'Avg_Playtime_2wk': int,
                  'Median_Playtime': int, 'Median_Playtime_2wk': int,
                  'Current_Price': int, 'Initial_Price': int, 'Discount': int}
    
    return convert_data_type(top_100_steam_spy_df,dtype_dict)

#Get Top 100 Applications by Player Count from Steam
def get_top_100_steam():  
    url = "http://store.steampowered.com/stats/"
    response = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(response,'lxml')
    game_list = soup.findAll("a",{'class': 'gameLink'})
    top_100_steam = []
    
    for game in game_list:
        top_100_steam.append([re.search('\d+',game.get('href')).group(), game.text])
    top_100_steam_df = pd.DataFrame(top_100_steam, columns = ['App_ID','App_Name'])
    
    dtype_dict = {'App_ID': int}
    
    return convert_data_type(top_100_steam_df,dtype_dict)

In [30]:
#Get Application Level Details from Steam Spy - API
def get_game_info_api_steamspy(app_id):
    url = 'http://steamspy.com/api.php?request=appdetails&appid=' + str(app_id)
    response = urllib.request.urlopen(url).read()
    game_info_api = json.loads(response.decode())
    game_info_api_df = pd.DataFrame.from_dict(game_info_api, orient='index')
    game_info_api_df.reset_index(level=0, inplace=True)
    game_info_api_df.columns = ['Metric','Value']
    game_info_api_df = pd.pivot_table(game_info_api_df, columns = 'Metric', aggfunc = np.unique)
    game_info_api_df.drop(['score_rank','userscore'], axis=1, inplace=True)
    game_info_api_df.columns = ['App_ID','Avg_Playtime_2wk','Avg_Playtime','Concurrent_Users','Developer','Discount',
                                'Genre','Initial_Price','Languages','Median_Playtime_2wk','Median_Playtime','App_Name',
                                'Negative','Owners','Positive','Current_Price','Publisher','Tags']
    game_info_api_df = game_info_api_df[['App_ID','App_Name','Developer','Publisher','Genre','Languages','Tags',
                                         'Positive','Negative','Owners','Concurrent_Users','Avg_Playtime','Avg_Playtime_2wk',
                                         'Median_Playtime','Median_Playtime_2wk','Current_Price','Initial_Price','Discount']]
   
    return game_info_api_df.reset_index(drop=True)

#Check index of required elements in Game Web Page
def check_availability(soup):
    
    soup_list = soup.find_all('strong')
    id_1 = [soup_list.index(i) for i in soup_list if 'Category:' in i]
    id_2 = [soup_list.index(i) for i in soup_list if 'Release date' in i]
    id_3 = [soup_list.index(i) for i in soup_list if 'Old userscore:' in i]
    id_4 = [soup_list.index(i) for i in soup_list if 'Metascore:' in i]
    id_5 = [soup_list.index(i) for i in soup_list if 'Followers' in i]
    id_6 = [soup_list.index(i) for i in soup_list if 'Peak concurrent players yesterday' in i]
    id_7 = [soup_list.index(i) for i in soup_list if 'Peak on Twitch yesterday' in i]
    id_8 = [soup_list.index(i) for i in soup_list if 'YouTube stats' in i]
    id_9 = [soup_list.index(i) for i in soup_list if 'Playtime in the last 2 weeks:' in i]
    id_10 = [soup_list.index(i) for i in soup_list if 'Playtime total:' in i]
    
    return [id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,id_9,id_10]

#Get Application Level Details from Steam Spy - Scraping
def get_game_info_scrape_steamspy(driver,app_id):
    driver.get('http://steamspy.com/app/' + str(app_id))
    sleep(10)
    soup = BeautifulSoup(driver.page_source,'lxml')
    
    info_metrics_df = scrape_info_steamspy(app_id,soup)
    Daily_CCU, Hourly_CCU, Daily_Twitch, Hourly_Twitch = scrape_charts_steamspy(app_id,soup)
    
    return (info_metrics_df, Daily_CCU, Hourly_CCU, Daily_Twitch, Hourly_Twitch)

In [31]:
def scrape_info_steamspy(app_id,soup):
    
    id_list = check_availability(soup)
    
    if (len(id_list[0])!=0):
        try:
            Category = soup.find_all('strong')[id_list[0][0]].next_sibling.strip()
        except:
            Category = ''
    else:
        Category = ''
    if (len(id_list[1])!=0):
        try:
            Release_Date = soup.find_all('strong')[id_list[1][0]].next_sibling.replace(':','').strip()
            Release_Date = re.search(r'\w{3}\s\d{1,2},\s\d{4}', Release_Date).group()
        except:
            Release_Date = ''
    else:
        Release_Date = ''
    if (len(id_list[2])!=0):
        try:
            User_Score = soup.find_all('strong')[id_list[2][0]].next_sibling.replace(':','').strip()
        except:
            User_Score = ''
    else:
        User_Score = ''
    if (len(id_list[3])!=0):
        try:
            Meta_Score = soup.find_all('strong')[id_list[3][0]].next_sibling.replace(':','').strip()
        except:
            Meta_Score = ''
    else:
        Meta_Score = ''
    if (len(id_list[4])!=0):
        try:
            Followers = soup.find_all('strong')[id_list[4][0]].next_sibling.replace(':','').replace(',','').strip()
        except:
            Followers = np.nan
    else:
        Followers = np.nan
    if (len(id_list[5])!=0):
        try:
            Peak_CCU_LD = soup.find_all('strong')[id_list[5][0]].next_sibling.replace(":","").replace(",","").strip()
        except:
            Peak_CCU_LD = np.nan
    else:
        Peak_CCU_LD = np.nan
    if (len(id_list[6])!=0):
        try:
            Peak_Twitch = soup.find_all('strong')[id_list[6][0]].next_sibling.replace(":","").replace(",","").strip()
            Peak_Twitch_Viewers_LD = re.findall('\d+',Peak_Twitch)[0]
            Peak_Twitch_Channels_LD = re.findall('\d+',Peak_Twitch)[1]
        except:
            Peak_Twitch_Viewers_LD = np.nan
            Peak_Twitch_Channels_LD = np.nan
    else:
        Peak_Twitch_Viewers_LD = np.nan
        Peak_Twitch_Channels_LD = np.nan
    if (len(id_list[7])!=0):
        try:
            Peak_YT = soup.find_all('strong')[id_list[7][0]].next_sibling.replace(":","").replace(",","").strip()
            Peak_YT_Views_LD = re.findall('\d+',Peak_YT)[0]
            Peak_YT_Comments_LD = re.findall('\d+',Peak_YT)[1]
        except:
            Peak_YT_Views_LD = np.nan
            Peak_YT_Comments_LD = np.nan
    else:
        Peak_YT_Views_LD = np.nan
        Peak_YT_Comments_LD = np.nan
    if (len(id_list[8])!=0):
        try:
            Playtime_L2W = soup.find_all('strong')[id_list[8][0]].next_sibling.strip()
            Avg_Playtime_L2W = re.findall('\d+:\d+',Playtime_L2W)[0]
            Median_Playtime_L2W = re.findall('\d+:\d+',Playtime_L2W)[1]
        except:
            Avg_Playtime_L2W = ''
            Median_Playtime_L2W = ''
    else:
        Avg_Playtime_L2W = ''
        Median_Playtime_L2W = ''
    if (len(id_list[9])!=0):
        try:
            Playtime_Total = soup.find_all('strong')[id_list[9][0]].next_sibling.strip()
            Avg_Playtime_Total = re.findall('\d+:\d+',Playtime_Total)[0]
            Median_Playtime_Total = re.findall('\d+:\d+',Playtime_Total)[1]
        except:
            Avg_Playtime_Total = ''
            Median_Playtime_Total = ''            
    else:
        Avg_Playtime_Total = ''
        Median_Playtime_Total = ''
    
    game_info_scrape = [[app_id,Category,Release_Date,User_Score,Meta_Score,Followers,Peak_CCU_LD,
                       Peak_Twitch_Viewers_LD,Peak_Twitch_Channels_LD,Peak_YT_Views_LD,Peak_YT_Comments_LD,
                       Avg_Playtime_L2W,Median_Playtime_L2W,Avg_Playtime_Total,Median_Playtime_Total]]
    
    game_info_scrape_df = pd.DataFrame(game_info_scrape,
                                       columns = ['App_ID','Category','Release_Date','User_Score','Meta_Score',
                                                  'Followers','Peak_CCU_LD','Peak_Twitch_Viewers_LD','Peak_Twitch_Channels_LD',
                                                 'Peak_YT_Views_LD','Peak_YT_Comments_LD','Avg_Playtime_L2W','Median_Playtime_L2W',
                                                 'Avg_Playtime_Total','Median_Playtime_Total'])
    return game_info_scrape_df

In [32]:
def scrape_charts_steamspy(app_id,soup):
    
    Daily_CCU_soup = soup.find('script', text=re.compile(r'data2ccu'))
    Hourly_CCU_soup = soup.find('script', text=re.compile(r'data2hccu'))
    Daily_Twitch_soup = soup.find('script', text=re.compile(r'datadtwitch'))
    Hourly_Twitch_soup = soup.find('script', text=re.compile(r'datahtwitch'))
        
    if (Daily_CCU_soup!=None):
        try:
            Daily_CCU_soup = Daily_CCU_soup.text.replace('\n','')
            Daily_CCU_soup = Daily_CCU_soup[Daily_CCU_soup.find('values')+6:len(Daily_CCU_soup)-3].replace('\"','').replace('}','').strip(':; ')
            Daily_CCU = pd.DataFrame(ast.literal_eval(Daily_CCU_soup.replace('#2B6A94','')), columns = ['Time_mili','CCU'])
            Daily_CCU['Time_mili'] = Daily_CCU['Time_mili']/(24*60*60*1000)
            Daily_CCU['Date'] = Daily_CCU['Time_mili'].apply(lambda x: timedelta(x) + datetime.datetime.strptime('01011970', "%d%m%Y").date())
            Daily_CCU.drop('Time_mili', axis = 1, inplace = True)
            Daily_CCU['App_ID'] = app_id
            Daily_CCU = Daily_CCU[['App_ID','Date','CCU']]
        except:
            Daily_CCU = pd.DataFrame({'App_ID': [app_id], 'Date': [np.nan], 'CCU': [np.nan]})
    else:
        Daily_CCU = pd.DataFrame({'App_ID': [app_id], 'Date': [np.nan], 'CCU': [np.nan]})    
    if (Hourly_CCU_soup!=None):
        try:
            Hourly_CCU_soup = Hourly_CCU_soup.text.replace('\n','')
            Hourly_CCU_soup = Hourly_CCU_soup[Hourly_CCU_soup.find('values')+6:len(Hourly_CCU_soup)-3].replace('\"','').replace('}','').strip(':; ')
            Hourly_CCU = pd.DataFrame(ast.literal_eval(Hourly_CCU_soup.replace('#2B6A94','')), columns = ['Time_mili','CCU'])
            Hourly_CCU['DateTime'] = Hourly_CCU['Time_mili'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
            Hourly_CCU.drop('Time_mili', axis = 1, inplace = True)
            Hourly_CCU['App_ID'] = app_id
            Hourly_CCU = Hourly_CCU[['App_ID','DateTime','CCU']]
        except:
            Hourly_CCU = pd.DataFrame({'App_ID': [app_id], 'DateTime': [np.nan], 'CCU': [np.nan]})
    else:
            Hourly_CCU = pd.DataFrame({'App_ID': [app_id], 'DateTime': [np.nan], 'CCU': [np.nan]})
    
    if (Daily_Twitch_soup!=None):
        try:
            Daily_Twitch_soup = Daily_Twitch_soup.text.replace('\n','')
            DT = Daily_Twitch_soup.replace('{','').split('}')
            DT_1 = DT[0]
            DT_2 = DT[1]
            DT_1 = DT_1[DT_1.find('values')+6:len(DT_1)].replace('\"','').strip(':; ')
            DT_2 = DT_2[DT_2.find('values')+6:len(DT_2)].replace('\"','').strip(':; ')
            DT_1_df = pd.DataFrame(ast.literal_eval(DT_1.replace('#2B6A94','')), columns = ['Time_mili','Twitch_Viewers'])
            DT_2_df = pd.DataFrame(ast.literal_eval(DT_2.replace('#2B6A94','')), columns = ['Time_mili','Twitch_Channels'])
                       
            DT_1_df['Time_mili'] = DT_1_df['Time_mili']/(24*60*60*1000)
            DT_1_df['Date'] = DT_1_df['Time_mili'].apply(lambda x: timedelta(x) + datetime.datetime.strptime('01011970', "%d%m%Y").date())
            DT_1_df.drop('Time_mili', axis = 1, inplace = True)
            DT_1_df['App_ID'] = app_id
            DT_1_df = DT_1_df[['App_ID','Date','Twitch_Viewers']]
            
            DT_2_df['Time_mili'] = DT_2_df['Time_mili']/(24*60*60*1000)
            DT_2_df['Date'] = DT_2_df['Time_mili'].apply(lambda x: timedelta(x) + datetime.datetime.strptime('01011970', "%d%m%Y").date())
            DT_2_df.drop('Time_mili', axis = 1, inplace = True)
            DT_2_df = DT_2_df[['Date','Twitch_Channels']]
            
            Daily_Twitch = pd.merge(DT_1_df, DT_2_df, on = 'Date', how = 'left')
            
        except:
            Daily_Twitch = pd.DataFrame({'App_ID': [app_id], 'Date': [np.nan], 'Twitch_Viewers': [np.nan], 
                                         'Twitch_Channels': [np.nan]})
    else:
        Daily_Twitch = pd.DataFrame({'App_ID': [app_id], 'Date': [np.nan], 'Twitch_Viewers': [np.nan], 
                                         'Twitch_Channels': [np.nan]})    
    if (Hourly_Twitch_soup!=None):
        try:
            Hourly_Twitch_soup = Hourly_Twitch_soup.text.replace('\n','')
            HT = Hourly_Twitch_soup.replace('{','').split('}')
            HT_1 = HT[0]
            HT_2 = HT[1]
            HT_1 = HT_1[HT_1.find('values')+6:len(HT_1)].replace('\"','').strip(':; ')
            HT_2 = HT_2[HT_2.find('values')+6:len(HT_2)].replace('\"','').strip(':; ')
            HT_1_df = pd.DataFrame(ast.literal_eval(HT_1), columns = ['Time_mili','Twitch_Viewers'])
            HT_2_df = pd.DataFrame(ast.literal_eval(HT_2), columns = ['Time_mili','Twitch_Channels'])
                      
            HT_1_df['DateTime'] = HT_1_df['Time_mili'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
            HT_1_df.drop('Time_mili', axis = 1, inplace = True)
            HT_1_df['App_ID'] = app_id
            HT_1_df = HT_1_df[['App_ID','DateTime','Twitch_Viewers']]
            
            HT_2_df['DateTime'] = HT_2_df['Time_mili'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
            HT_2_df.drop('Time_mili', axis = 1, inplace = True)
            HT_2_df = HT_2_df[['DateTime','Twitch_Channels']]
            
            Hourly_Twitch = pd.merge(HT_1_df, HT_2_df, on = 'DateTime', how = 'left')
            
        except:
            Hourly_Twitch = pd.DataFrame({'App_ID': [app_id], 'DateTime': [np.nan], 'Twitch_Viewers': [np.nan], 
                                         'Twitch_Channels': [np.nan]})
    else:
        Hourly_Twitch = pd.DataFrame({'App_ID': [app_id], 'DateTime': [np.nan], 'Twitch_Viewers': [np.nan], 
                                         'Twitch_Channels': [np.nan]})
    
    return (Daily_CCU, Hourly_CCU, Daily_Twitch, Hourly_Twitch)

In [37]:
##########################################################
#                   Main Section Starts                  #
##########################################################

#Assign path of chromedriver
driver_path = 'D:/Sayantan_C'

#Getting Top 100 games from Steam Spy and Steam
top_100_steam_spy_df = get_top_100_steamspy()
top_100_steam_df = get_top_100_steam()

#Creating master Game list to extract Steam Spy information
master_game_list = pd.DataFrame(top_100_steam_spy_df['App_ID'].append(top_100_steam_df['App_ID'])
                                .unique().tolist(), columns = ['App_ID'])

In [33]:
##########################################################
#               Get API Info - Steam Spy                 #
##########################################################

#Creating Empty DataFrame for SteamSpy API Info for all Games
master_game_info_api_df = pd.DataFrame(columns = ['App_ID','App_Name','Developer','Publisher','Genre','Languages','Tags',
                                         'Positive','Negative','Owners','Concurrent_Users','Avg_Playtime','Avg_Playtime_2wk',
                                         'Median_Playtime','Median_Playtime_2wk','Current_Price','Initial_Price','Discount'])
for app_id in tqdm(master_game_list['App_ID']):
    app_info = get_game_info_api_steamspy(app_id)
    master_game_info_api_df = master_game_info_api_df.append(app_info)

dtype_dict = {'App_ID': int, 'Positive': int, 'Negative': int, 'Concurrent_Users':int, 'Avg_Playtime': int,
              'Avg_Playtime_2wk': int, 'Median_Playtime': int, 'Median_Playtime_2wk': int,
              'Current_Price': int, 'Initial_Price': int, 'Discount': int}

master_game_info_api_df = convert_data_type(master_game_info_api_df.reset_index(drop=True),dtype_dict)

C:\Users\sayantan\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [38]:
##########################################################
#              Get Scraped Info - Steam Spy              #
##########################################################

#Creating Empty DataFrame for SteamSpy Scrape Info for all Games
master_game_info_scrape_df = pd.DataFrame(columns = ['App_ID','Category','Release_Date','User_Score','Meta_Score',
                                                     'Followers','Peak_CCU_LD','Peak_Twitch_Viewers_LD','Peak_Twitch_Channels_LD',
                                                     'Peak_YT_Views_LD','Peak_YT_Comments_LD','Avg_Playtime_L2W',
                                                     'Median_Playtime_L2W','Avg_Playtime_Total','Median_Playtime_Total'])

master_daily_CCU_df = pd.DataFrame(columns = ['App_ID','Date','CCU'])
master_hourly_CCU_df = pd.DataFrame(columns = ['App_ID','DateTime','CCU'])
master_daily_Twitch_df = pd.DataFrame(columns = ['App_ID','Date','Twitch_Viewers','Twitch_Channels'])
master_hourly_Twitch_df = pd.DataFrame(columns = ['App_ID','DateTime','Twitch_Viewers','Twitch_Channels'])

driver = webdriver.Chrome(driver_path + '/chromedriver.exe')
driver.implicitly_wait(5)

for app_id in tqdm(master_game_list['App_ID']):
    try:
        app_info, Daily_CCU, Hourly_CCU, Daily_Twitch, Hourly_Twitch = get_game_info_scrape_steamspy(driver, app_id)
    except TimeoutException as e:
        continue
    master_game_info_scrape_df = master_game_info_scrape_df.append(app_info)
    master_daily_CCU_df = master_daily_CCU_df.append(Daily_CCU)
    master_hourly_CCU_df = master_hourly_CCU_df.append(Hourly_CCU)
    master_daily_Twitch_df = master_daily_Twitch_df.append(Daily_Twitch)
    master_hourly_Twitch_df = master_hourly_Twitch_df.append(Hourly_Twitch)

driver.quit()

#dtype_dict = {'App_ID': int, 'Followers': int, 'Peak_CCU_LD': int, 'Peak_Twitch_Viewers_LD': int, 'Peak_Twitch_Channels_LD': int,
#              'Peak_YT_Views_LD': int, 'Peak_YT_Comments_LD': int}

#master_game_info_scrape_df = convert_data_type(master_game_info_scrape_df.reset_index(drop=True),dtype_dict)